In [4]:
# Librerie base
import pandas as pd
import numpy as np

# Machine Learning
import joblib
from sklearn.preprocessing import FunctionTransformer

# Management di file
import os

# Flask
from flask import Flask, render_template, request

In [6]:
# Definizione dei percorsi e dei parametri globali
DATASET_BASE_PATH: str = './datasets'
COMBATS_PATH: str = f"{DATASET_BASE_PATH}/combats.csv"
TYPE_CHART_PATH: str = f"{DATASET_BASE_PATH}/type_chart.csv"
POKEMON_PATH: str = f"{DATASET_BASE_PATH}/pokemon.csv"
MODEL_DIR: str = './models'

In [7]:
def load_pokemon_data(combats_path: str, pokemon_path: str) -> pd.DataFrame:
   df_combats = pd.read_csv(combats_path)
   df_pokemon = pd.read_csv(pokemon_path, index_col="#").fillna("None")
   # Prerprocessing del campo winner per avere 0 se vince il primo Pokemon e 1 se vince il_So
   df_combats["Winner"] = (df_combats["Winner"] != df_combats["First_pokemon"]).astype(int)
   # Merge first and second Pokémon stats
   return df_combats.merge(
      df_pokemon.add_suffix("_F"), how="left",
      left_on="First_pokemon", right_index=True
   ).merge(
      df_pokemon.add_suffix("_S"), how="left",
      left_on="Second_pokemon", right_index=True
   ).drop(
      columns=["First_pokemon", "Second_pokemon"]
   )

In [8]:
# Caricamento dei DataFrame
df_type_chart = pd.read_csv(TYPE_CHART_PATH).fillna("None")
df_pokemon = pd.read_csv(POKEMON_PATH, index_col="#").fillna("None")
df_combats: pd.DataFrame = load_pokemon_data(COMBATS_PATH, POKEMON_PATH)

In [9]:
from utils import function_transformer
data_transformer: FunctionTransformer = FunctionTransformer(func=function_transformer, kw_args={'dataframe_type_chart': df_type_chart}).set_output(transform="pandas")

In [10]:
app: Flask = Flask(__name__, template_folder='webpage')

models = os.listdir(MODEL_DIR)
model = joblib.load(os.path.join(MODEL_DIR, models[0]))

@app.route('/', methods=['GET', 'POST'])
def one_v_one_battle():
    pokemon_first: str | None = None
    pokemon_second: str | None = None
    winner: str | None = None
    confidence: float | None = None
    if request.method == 'POST':
        if request.form.get('change_model'):
            model_filename: str = request.form.get('model_file')
            if model_filename in models:
                global model
                model = joblib.load(os.path.join(MODEL_DIR, model_filename))
        if request.form.get('predict_fight'):
            pokemon_first = request.form.get('pokemon_first')
            pokemon_second = request.form.get('pokemon_second')
            first_data = df_pokemon.loc[df_pokemon['Name'] == pokemon_first].add_suffix(suffix='_F')
            second_data = df_pokemon.loc[df_pokemon['Name'] == pokemon_second].add_suffix(suffix='_S')
            winner_id = model.predict(first_data.merge(second_data, how='cross'))
            winner = pokemon_first if winner_id == 0 else pokemon_second
            confidence = np.max(model.predict_proba(first_data.merge(second_data, how='cross')), axis=1)[0]* 100
    return render_template('1v1_battle.html', models=models, selected_pokemons=[pokemon_first, pokemon_second], pokemon_choices=df_pokemon['Name'], winner=winner, confidence=confidence)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
